In [236]:
# 1. Load the Lib
import torch
import os
import time
import pandas as pd
import torch.nn as nn
import numpy as np
import seaborn as sns # Use for plot
import torch.optim as optim 
import torch.nn.functional as func
import matplotlib.pyplot as plt # Use for plot
import sklearn 
import torchvision

from torchvision import datasets, transforms, models, utils
from torchsummary import summary # Visualizing Training Process
from torch.utils.data import DataLoader

# Use for Machine Learning
from mlxtend.plotting import plot_confusion_matrix

from PIL import Image # Open/Read an Image


In [237]:
# 2. Define the Hyperparameters\
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCH = 10

In [238]:
# 3. Image Transformation
image_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(300), # Randomly Crop 300x300 images
        # Rotation
        # ColorJitter
        transforms.RandomHorizontalFlip(), # Horizontally Flip the image
        transforms.CenterCrop(256), # Crop 256x256 image from cener
        transforms.ToTensor(), # Transform images to Tensor
        transforms.Normalize([.485, .456, .406],[.229, .224, .225]) # Normalization for RGB
    ]),
    
    'val': transforms.Compose([
        transforms.Resize(300), # Resize the image to 300x300
        transforms.RandomHorizontalFlip(), # Horizontally Flip the image
        transforms.ToTensor(), # Transform images to Tensor
        transforms.Normalize([.485, .456, .406],[.229, .224, .225])
    ]),
    
    'test':([
        transforms.Resize(300), # Resize the image to 300x300
        transforms.RandomHorizontalFlip(), # Horizontally Flip the image
        transforms.ToTensor(), # Transform images to Tensor
        transforms.Normalize([.485, .456, .406],[.229, .224, .225]) # Normalization for RGB
    ])
}   

In [239]:
#  4. Load-in Dataset
BATCH_SIZE = 128
file_dir = '/Users/xinghaozhou/Desktop/DL/chest_xray/'

    # 5.1 Train/Val/Test dir 
train_data_dir = file_dir + 'train/'
val_data_dir= file_dir + 'val/'
test_data_dir = file_dir + 'test/'

    # 5.2 Set train/val/test datasets
datasets = {
    'train': torchvision.datasets.ImageFolder(train_data_dir, transform = image_transforms['train']),
    'val': torchvision.datasets.ImageFolder(val_data_dir, transform = image_transforms['val']),
    'test':torchvision.datasets.ImageFolder(test_data_dir, transform = image_transforms['test'])   
}

    # 5.3 Load in Images for different datsets by the batch_size
dataloader = {
    'train': DataLoader(datasets['train'], batch_size = BATCH_SIZE, shuffle = True),
    'val' : DataLoader(datasets['val'], batch_size = BATCH_SIZE, shuffle = True),
    'test': DataLoader(datasets['test'], batch_size = BATCH_SIZE, shuffle = True)    
}

    # 5.4 Get the Label name and make it as a "DICT"
LABEL = dict((v, k) for k, v in datasets['train'].class_to_idx.items())
LABEL

In [240]:
# APPENDIX. Train set info
dataloader['train'].dataset
dataloader['val'].batch_size
dataloader['val'].dataset.root

In [241]:
# 5. Normal Images
image_normal = os.listdir(os.path.join(dataloader['train'].dataset.root,'NORMAL'))
image_pneumonia = os.listdir(os.path.join(dataloader['train'].dataset.root,'Pneumonia'))

In [242]:
# 6. Log writter
from torch.utils.tensorboard import SummaryWriter

log_dir = '/Users/xinghaozhou/Desktop/DL/chest_xray/logdir/'

def tb_writer():
    timestr = time.strftime("%Y%m%d__%H%M%S")
    writer = SummaryWriter(log_dir+timestr)
    return writer
writer = tb_writer()


In [243]:
# 7. Setting for showing images
images, label = next(iter(dataloader['train']))
def imshow(img):
    img = img /2 +0.5 # Unnormalize
    npimg = img.numpy() # Tensor -> Numpy
    plt.imshow(np.transpose(npimg, (1, 2, 0))) # Change the Channel Order Reason
    # Reason: plt and Numpy represent 3-dimension in (x,y,z) format whereas PyTorch represents is as (z,x,y) format
    plt.show() 
    
grid = utils.make_grid(images) # Make grid that splits the images
imshow(grid) # Show the grid

writer.add_image('X-Ray Grid: ', grid, 0) # add_image(tag, image_tensor, global_step ...)
writer.flush()

In [244]:
# 8. Misclassified Images
def misclassified_images(pred, writer, label, images, output, epoch, counts=10):
    misclassified = (pred != label.data ) # Determine if it is missclassified
    for index, image_tensor in enumerate(images[misclassified][:counts]):
        img_name = 'Epoch:{}-->Predict:{}-->Actual:{}'.format(epoch, LABEL[pred[misclassified].tolist()[index]],
                                                              LABEL[label.data[misclassified].tolist()[index]])
        writer.add_image(img_name, images_tensor, epoch) 

In [245]:
# 9. Modification of Pooling Layer
# Q: Why we have this? A: While training the models, some params might change. We need this to adapt to new params
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, size = None):
        super(AdaptiveConcatPool2d,self).__init__()
        size = size or (1,1) # Want it to be 1x1 filter
        self.pool_one = nn.AdaptiveAvgPool2d(size) # Modifying 1st Pooling Layer 
        self.pool_two = nn.AdaptiveMaxPool2d(size) # Modifying 2nd Pooling Layer
        
    def forward(self, x):
        return torch.cat([self.pool_one(x), self.pool_two(x)], dim=1) # Concatenate those two new layers seperately

In [246]:
# 10. Transfer Learning
def get_model():
    model = models.resnet50(pretrained=True) # Using Pre-trained model: Resnet-50
    
    for param in model.parameters(): 
        param.requires_grad = False    # Freeze part of the model
    
    model.avgpool = AdaptiveConcatPool2d() # Modifying AvgPooling Layer to Adaptiive Pooling Layer 
    model.fc = nn.Sequential( # Container for FC Layer
        nn.Flatten(),   # Flattern to 1-dim
        nn.BatchNorm1d(4096), # Normalization
        nn.Dropout(0.5), # Dropout 
        nn.Linear(4096,512), # Input: 4096 with 1-dim   Output: 512 with 1-dim
        nn.ReLU(), # ReLu Activation Func
        nn.BatchNorm1d(512), # Normalizationi
        nn.Linear(512,2), # Input: 512 with 1-dim    Output: 2 stands for Pneumonia/Normal
        nn.LogSoftmax(dim=1) # Softmax Func to get Prob
    )
    return model # Return the model
    

In [247]:
# 11. Define the Train model
def train_val(model, device, criterion, train_loader, val_loader, optimizer, epoch, writer): # Writer for outputing result into LOG
    model.train() # Train model
    total_loss = 0.0 # Loss rate
    val_loss = 0.0 # Val rate
    val_accu = 0.0 # Val accuracy
    
    for batch_index, (data, label) in enumerate(train_loader): # Iterate over each batch
        data, label = data.to(device), label.to(device) # Send data/label to device
        optimizer.zero_grad() # Initialize gradient to 0 
        output = model(data) # Foward propagation
        loss = criterion(data, label) # Calculate the loss rate 
        loss.backward() # Backpropagation 
        total_loss += loss.item() * image.size(0) # Accumulate the total_loss 
        # Reason for why we use image.size(0) -> image.size(0) gives us the current batch_size
        # Reason for mul -> since loss.item() gives us the average batch loss 
        
    train_loss = total_loss / len(train_loader.dataset) # Get the average training loss
    writer.add_scalar("Train loss:", train_loss, epoch) # Write-in the Average Loss rate into Log
    writer.flush() # Write log into the Disk
    
    model.eval()
    with torch.no_grad():
        for data, label in val_loader:
            data, label = data.to(device), label.to(device) # Send data/label to device
            output = model(data) # Foward propagation
            loss = criterion(data, label) # Calculate the loss rate 
            loss.backward() # Backpropagation 
            val_loss += loss.item() * image.size(0) # Accumulate the total_loss 
            pred = output.argmax(dim=1) # Pred is the most possible predicition
            correct = pred.eq(label.veiw_as(pred)) # Accumulate the correct, return as tensor[True, False, False.....]
            # Reason --> Because the batch contains (BATCH_SIZE) images; therefore, the tensor has 128 boolean
            accuracy = torch.mean(correct.type(torch.FloatTensor)) # Calculates (Float) (number of "True" in tensor/ tensor size)
            val_accu += accuracy.item * image.size(0)
        val_loss = val_loss / len(val_loader.dataset)
        val_accu = val_accu / len(val_loader.dataset)
        
 # Return the Average Loss Rate
    return train_loss, val_loss, val_accu

In [248]:
# 12. Define the Test model
def test_model(model, device, criterion, test_loader, writer, epoch):
    model.eval() # Test model
    # Initialization 
    correct = 0.0 
    test_loss = 0.0 
    
    with torch.zero_grad():
        for batch_id, (iamges, labels) in enumerate(test_loader):
            data, label = data.to(device), label.to(device) # Send data/label to device
            output = model(data)  # Foward propagation
            loss = criterion(data, label).item() # Calculate the loss rate 
            test_loss += loss.item() # Accumulate the total_loss 
            pred = output.argmax(dim=1)  # Pred is the most possible predicition
            correct = pred.eq(label.veiw_as(pred)).sum().item() # Get the num that we predict correctly
            misclassified_images(pred, writer, label, images, output, epoch, counts=10)
            
            
        avg_loss /= len(test_loader) # Calculate the Average Loss Rate
        accuracy = 100 * correct / len(test_loader) # Calculate the Accuracy

    writer.add_scalar("Test Loss:", total_loss, epoch)
    writer.add_scalar("Test Accuracy:", Accuracy, epoch)
    writer.flush()
    return test_loss, accuracy

In [249]:
# 13. Define the Train process
model = get_model().to(DEVICE)
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr =0.003)

def train_epochs(model, device, dataloader, criterion, epoch, optimizer, writer):
    # Output INFO
    print("{0:>15} | {1:>15} | {2:>15} | {3:>15} | {4:>15} | {5:>15}".format("Epoch", "Train Loss", "Val Loss", "Val Accuracy", "Test Loss", "Test Accuracy"))
    
    # Save the BEST model
    best_loss = np.inf
    
    for epoch in range(epoch):
        # Train dataset and Return values of "train_loss", "val_loss", "val_accu"
        train_loss, val_loss, val_accu = train_val(model, device, criterion, dataloader['train'], dataloader['val'], optimizer, epoch, writer)
        # Test dataset and Return values of "total_loss" and "accuracy"
        test_loss, accuracy = test_model(model, device, criterion, dataloader['test'], writer, epoch)
        # Update the new best_loss if it is less than the previous one
        if best_loss < test_loss:
            best_loss = test_loss
            # Save the model
            torch.save(model.state_dict(), 'model.pth')
        # Output the Result
        print("{0:>15} | {1:>15} | {2:>15} | {3:>15} | {4:>15} | {5:>15}".format("Epoch", "Train Loss", "Val Loss", "Val Accuracy", "Test Loss", "Test Accuracy"))
        writer.flush()

In [250]:
model = get_model().to(DEVICE) # Get the model
criterion = nn.NLLLoss() # Loss Function
optimizer = optim.Adam(model.parameters()) # Optimizer Used
train_epochs(model, DEVICE, dataloaders, criterion, optimizer, EPOCH, writer)
writer.close() 